In [ ]:
from casadi import *
import math_tools as mt
import constants as con

#################################################################
p_i = 21837080.052835
f_i = 0
g_i = 0
h_i = -0.25396764647494
k_i = 0
L_i = pi
w_i = 1

e_f = 0.73550320568829 # final eccentricity 
Isp = 450 # specific impulse

tau = -50 # -99 < tau < 0
mu = 1.407645794e16
J2 = 1082.639e-6
J4 = -1.608e-6
p_f = 40007346.015232 # final p
tanif2 = 0.6176125876099
g0 = 32.174 # ft/s^2
T = 4.446618e-3 # max thrust
Re = 20925662.73

#################################################################



# States
Nstates = 7
p = MX.sym('p')
f = MX.sym('f')
g = MX.sym('g')
h = MX.sym('h')
k = MX.sym('k')
L = MX.sym('L')
w = MX.sym('w')
y = vertcat(p,f,g,h,k,L,w)

# Controls
Ncontrols = 3
ur = MX.sym('ur')
ut = MX.sym('ut')
un = MX.sym('un')
u = vertcat(ur,ut,un)

# Equations of Motion
q = 1 + (f*cos(L)) + (g*sin(L))
chi = sqrt(h**2 + k**2)
ss = 1 + chi**2
alpha = h**2 - k**2

delta_T = g0 * T * (1 + 0.01*tau)  * u 
delta = delta_T

pdot = (2 * p * sqrt(p/mu) / q) * delta_T[1]
fdot = ((sqrt(p/mu)*sin(L))*delta[0]) + ((sqrt(p/mu)*(((q + 1) * cos(L)) + f) / q)*delta[1]) + ((-sqrt(p/mu) * (g/q) * ((h*sin(L)) - (k*cos(L))))*delta[2])
gdot = ((-sqrt(p/mu)*cos(L))*delta[0]) + ((sqrt(p/mu)*(((q + 1) * sin(L)) + g) / q)*delta[1]) + ((sqrt(p/mu) * (f/q) * ((h*sin(L)) - (k*cos(L))))*delta[2]) 
hdot = (sqrt(p/mu) * ss * cos(L) / (2*q))*delta[2]
kdot = (sqrt(p/mu) * ss * sin(L) / (2*q))*delta[2]
Ldot = (sqrt(p/mu) * (1/q) * ((h*sin(L)) - (k*cos(L))))*delta[2] + (sqrt(mu*p) * ((q/p)*(q/p)))
wdot = -T * (1 + 0.01*tau) / Isp
ydot = vertcat(pdot,fdot,gdot,hdot,kdot,Ldot,wdot)


r = p/q
r_vec = MX.sym('r_vec',3)
r_vec[0] = (r/ss) * ((cos(L)) + (alpha*cos(L)) + (2*h*k*sin(L)))
r_vec[1] = (r/ss) * ((sin(L)) - (alpha*sin(L)) + (2*h*k*cos(L)))
r_vec[2] = (2*r/ss) * ((h*sin(L)) - (k*cos(L)))

v_vec = MX.sym('v_vec',3)
v_vec[0] = (-1/ss) * sqrt(mu/p) * (sin(L) + (alpha*sin(L)) - (2*h*k*cos(L)) + g - (2*f*h*k) + (alpha*g))
v_vec[1] = (-1/ss) * sqrt(mu/p) * (-cos(L) + (alpha*cos(L)) + (2*h*k*sin(L)) - f + (2*g*h*k) + (alpha*f))
v_vec[2] = (2/ss) * sqrt(mu/p) * ((h*cos(L)) + (k*sin(L)) + (f*h) + (g*k))


# Continous time dynamics
F = Function('F', [y, u], [ydot], ['y', 'u'], ['ydot'])

# Discritization
Ngridpoints = 1000

# Problem definition
opti = Opti()
tf = 90000
dt = tf/Ngridpoints
Y = opti.variable(Nstates,Ngridpoints)
Ymid = opti.variable(Nstates,Ngridpoints-1)
U = opti.variable(Ncontrols,Ngridpoints)
Umid = opti.variable(Ncontrols,Ngridpoints-1)


opti.set_initial(Y[0,:], p_i)
opti.set_initial(Y[1,:], f_i)
opti.set_initial(Y[2,:], g_i)
opti.set_initial(Y[3,:], h_i)
opti.set_initial(Y[4,:], L_i)
opti.set_initial(Y[5,:], w_i)
opti.set_initial(Ymid[0,:], p_i)
opti.set_initial(Ymid[1,:], f_i)
opti.set_initial(Ymid[2,:], g_i)
opti.set_initial(Ymid[3,:], h_i)
opti.set_initial(Ymid[4,:], L_i)
opti.set_initial(Ymid[5,:], w_i)

# State Constraints
opti.subject_to(vec(Y[0,:]) >= 0.1 * p_i)
opti.subject_to(vec(Ymid[0,:]) >= 0.1 * p_i)
opti.subject_to(vec(Y[0,:]) <= 5 * p_f)
opti.subject_to(vec(Ymid[0,:]) <= 5 * p_f)
opti.subject_to(vec(Y[1:5,:]) >= -1)
opti.subject_to(vec(Ymid[1:5,:]) >= -1)
opti.subject_to(vec(Y[1:5,:]) <= 1)
opti.subject_to(vec(Ymid[1:5,:]) <= 1)
opti.subject_to(vec(Y[5,:]) >= pi)
opti.subject_to(vec(Ymid[5,:]) >= pi)
opti.subject_to(vec(Y[5,:]) <= 34*pi)
opti.subject_to(vec(Ymid[5,:]) <= 34*pi)
opti.subject_to(vec(Y[6,:]) >= 0.001)
opti.subject_to(vec(Ymid[6,:]) >= 0.001)
opti.subject_to(vec(Y[6,:]) <= 1.01)
opti.subject_to(vec(Ymid[6,:]) <= 1.01)

# Initial Conditions
opti.subject_to(Y[0,0] == p_i)
opti.subject_to(Y[1,0] == f_i)
opti.subject_to(Y[2,0] == g_i)
opti.subject_to(Y[3,0] == h_i)
opti.subject_to(Y[4,0] == k_i)
opti.subject_to(Y[5,0] == L_i)
opti.subject_to(Y[6,0] == w_i)

# Final Conditions
opti.subject_to(Y[0,-1] == p_f)
opti.subject_to(Y[1,-1]**2 + Y[2,-1]**2 == e_f**2) # final eccentricity constraint
opti.subject_to(Y[3,-1]**2 + Y[4,-1]**2 == tanif2**2) # final inclination constraint
opti.subject_to(Y[1,-1]*Y[3,-1] + Y[2,-1]*Y[4,-1] == 0)
opti.subject_to(Y[2,-1]*Y[3,-1] - Y[4,-1]*Y[1,-1] <= 0)

# Control Constraints
opti.subject_to(vec(U) >= -1.1)
opti.subject_to(vec(Umid) >= -1.1)
opti.subject_to(vec(U) <= 1.1)
opti.subject_to(vec(Umid) <= 1.1)


# Hermite Simpson Constraints
for k in range(1,Ngridpoints-1):
    opti.subject_to(vec(Ymid[:,k-1] - (0.5 * (Y[:,k] - Y[:,k-1])) - ((dt/8)*(F(Y[:,k-1],U[:,k-1]) - F(Y[:,k],U[:,k])))) == 0)          # Hermite
    opti.subject_to(vec(Y[:,k] - Y[:,k-1] - ((dt/6)*(F(Y[:,k],U[:,k]) - (4 * F(Ymid[:,k-1],Umid[:,k-1])) - F(Y[:,k-1],U[:,k-1])))) == 0 ) # Simpson



# Objective
opti.minimize(-Y[6,-1])

# Solver
opti.solver('ipopt')

# Solution
sol = opti.solve()



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   230554
Number of nonzeros in inequality constraint Jacobian.:    39984
Number of nonzeros in Lagrangian Hessian.............:    63912

Total number of variables............................:    19990
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

  78r 7.6102388e-04 2.17e+07 1.70e+14   1.7 1.55e+01  12.5 3.21e-01 1.00e+00h  1
  79r 7.6102388e-04 2.17e+07 6.32e+13   1.7 1.06e+01  12.9 1.00e+00 2.32e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r 7.6102388e-04 2.17e+07 2.59e+14   1.7 3.71e+01  14.2 1.00e+00 7.40e-04h  1
  81r 7.6102388e-04 2.17e+07 6.29e+13   1.7 3.75e+01  13.8 0.00e+00 2.62e-07R  6
  82r 7.6102388e-04 2.17e+07 2.00e+11   1.7 2.86e-11  14.2 9.90e-01 1.00e+00f  1
  83r 7.6102388e-04 2.17e+07 2.00e+09   1.7 1.88e-10  13.7 9.90e-01 1.00e+00f  1
  84r 7.6102388e-04 2.17e+07 2.00e+07   1.7 1.03e-09  13.2 9.90e-01 1.00e+00f  1
  85r 7.6102388e-04 2.17e+07 1.98e+05   1.7 1.11e-09  12.8 9.90e-01 1.00e+00f  1
  86r 7.6102388e-04 2.17e+07 4.59e+03   1.7 2.43e-09  12.3 1.00e+00 1.00e+00f  1
  87r 7.6102388e-04 2.17e+07 4.42e+03   1.7 7.03e-09  11.8 1.00e+00 1.00e+00f  1
  88r 7.6102388e-04 2.17e+07 4.39e+03   1.7 2.10e-08  11.3 1.00e+00 1.00e+00f  1
  89r 7.6102388e-04 2.17e+07

 173r 7.4616214e-04 2.19e+07 2.48e+03   1.0 4.24e+00   2.4 5.86e-01 2.55e-01f  1
 174r 7.4545432e-04 2.19e+07 4.81e+04   1.0 1.59e+00   2.8 3.95e-01 3.80e-01f  1
 175r 7.4536695e-04 2.18e+07 2.19e+04   1.0 7.52e-02   4.1 1.00e+00 1.00e+00f  1
 176r 7.4533919e-04 2.18e+07 2.87e+05   1.0 1.89e+00   3.7 2.27e-01 1.06e-01f  1
 177r 7.4533008e-04 2.17e+07 1.58e+06   1.0 3.82e-02   5.0 6.51e-01 7.42e-01f  1
 178r 7.4532950e-04 2.17e+07 5.51e+05   1.0 6.43e-03   6.3 1.00e+00 1.00e+00f  1
 179r 7.4532778e-04 2.17e+07 7.76e+04   1.0 2.24e-03   5.8 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180r 7.4532259e-04 2.17e+07 1.04e+03   1.0 4.42e-03   5.4 1.00e+00 1.00e+00f  1
 181r 7.4530704e-04 2.17e+07 1.00e+03   1.0 1.33e-02   4.9 1.00e+00 1.00e+00f  1
 182r 7.4526040e-04 2.17e+07 1.00e+03   1.0 3.98e-02   4.4 1.00e+00 1.00e+00f  1
 183r 7.4512050e-04 2.17e+07 1.00e+03   1.0 1.19e-01   3.9 1.00e+00 1.00e+00f  1
 184r 7.4470104e-04 2.17e+07

 267r 7.3328845e-04 2.20e+07 6.05e+03   1.0 5.80e+00   3.6 2.01e-03 1.69e-03f  1
 268r 7.3325568e-04 2.20e+07 1.27e+05   1.0 8.86e+00   3.1 3.02e-02 3.61e-02f  1
 269r 7.3321309e-04 2.21e+07 1.70e+06   1.0 4.89e-02   4.4 4.16e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270r 7.3321232e-04 2.21e+07 1.24e+06   1.0 4.15e-02   5.8 3.82e-01 3.86e-01f  1
 271r 7.3321209e-04 2.21e+07 9.19e+05   1.0 2.94e-02   6.2 7.48e-01 3.05e-01f  1
 272r 7.3321186e-04 2.21e+07 8.29e+05   1.0 1.88e-01   5.7 1.86e-01 1.04e-01f  1
 273r 7.3321185e-04 2.21e+07 8.21e+05   1.0 1.31e+00   6.1 4.22e-02 9.38e-03f  1
 274r 7.3321166e-04 2.21e+07 7.64e+05   1.0 2.94e+00   5.7 2.69e-01 7.36e-02f  1
 275r 7.3321163e-04 2.21e+07 2.61e+06   1.0 2.10e+00   6.1 2.09e-02 3.75e-02f  1
 276r 7.3321160e-04 2.21e+07 2.13e+06   1.0 2.85e-01   6.5 1.67e-01 8.47e-02f  1
 277r 7.3321147e-04 2.21e+07 1.29e+06   1.0 1.62e+00   6.0 4.09e-02 1.20e-01f  1
 278r 7.3321147e-04 2.21e+07

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 360r 7.2520003e-04 2.22e+07 2.38e+05   1.0 3.31e+01   6.2 9.03e-04 6.14e-04h  1
 361r 7.2520003e-04 2.22e+07 2.45e+05   1.0 4.42e+01   5.7 7.03e-05 6.38e-04h  1
 362r 7.2520003e-04 2.22e+07 2.80e+05   1.0 3.46e+01   6.1 1.40e-03 7.77e-04h  1
 363r 7.2520003e-04 2.22e+07 2.84e+05   1.0 5.62e+01   5.6 4.84e-05 8.17e-04h  1
 364r 7.2520003e-04 2.22e+07 2.92e+05   1.0 3.50e+01   6.1 1.96e-03 1.84e-04h  1
 365r 7.2520003e-04 2.22e+07 2.90e+05   1.0 7.12e+01   5.6 2.58e-04 4.87e-04h  1
 366r 7.2520003e-04 2.22e+07 2.92e+05   1.0 3.59e+01   6.0 1.51e-03 7.53e-05h  1
 367r 7.2520003e-04 2.22e+07 2.89e+05   1.0 9.45e+01   5.5 4.04e-05 2.51e-04f  1
 368r 7.2520003e-04 2.22e+07 3.01e+05   1.0 3.68e+01   6.0 2.61e-04 4.61e-04h  1
 369r 7.2520003e-04 2.22e+07 3.07e+05   1.0 2.24e+02   5.5 6.53e-05 2.29e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 7.2520002e-04 2.22e+07

...

1301r 5.8564771e-04 3.15e+07 7.92e+08   1.0 7.17e-06  13.6 1.00e+00 5.00e-01h  2
1302r 5.8564772e-04 3.15e+07 5.45e+07   1.0 3.79e-06  13.2 1.00e+00 1.00e+00h  1
1303r 5.8564771e-04 3.15e+07 3.39e+06   1.0 7.07e-07  12.7 1.00e+00 1.00e+00h  1
1304r 5.8564771e-04 3.15e+07 4.50e+04   1.0 2.82e-08  12.2 1.00e+00 1.00e+00f  1
1305r 5.8564771e-04 3.15e+07 7.84e+03   1.0 1.47e-08  11.7 1.00e+00 1.00e+00f  1
1306r 5.8564771e-04 3.15e+07 6.56e+03   1.0 3.69e-08  11.2 1.00e+00 1.00e+00f  1
1307r 5.8564771e-04 3.15e+07 5.60e+03   1.0 9.46e-08  10.8 1.00e+00 1.00e+00f  1
1308r 5.8564771e-04 3.15e+07 5.84e+03   1.0 2.96e-07  10.3 1.00e+00 1.00e+00f  1
1309r 5.8564771e-04 3.15e+07 5.83e+03   1.0 8.87e-07   9.8 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1310r 5.8564771e-04 3.15e+07 5.83e+03   1.0 2.66e-06   9.3 1.00e+00 1.00e+00f  1
1311r 5.8564771e-04 3.15e+07 5.83e+03   1.0 7.98e-06   8.9 1.00e+00 1.00e+00f  1
1312r 5.8564771e-04 3.15e+07

1395r 5.8332534e-04 3.15e+07 1.95e+03   1.0 8.28e-02   4.4 1.00e+00 9.36e-01f  1
1396r 5.8330898e-04 3.15e+07 1.97e+03   1.0 3.02e-01   3.9 3.98e-01 1.41e-01f  1
1397r 5.8326549e-04 3.15e+07 2.02e+03   1.0 9.72e-02   4.3 1.00e+00 1.00e+00f  1
1398r 5.8314537e-04 3.15e+07 3.64e+03   1.0 4.35e-01   3.9 4.94e-01 9.21e-01f  1
1399r 5.8313668e-04 3.15e+07 3.65e+03   1.0 1.55e+00   3.4 1.48e-01 2.22e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1400r 5.8303911e-04 3.15e+07 2.52e+03   1.0 2.66e-01   3.8 1.24e-01 6.65e-01f  1
1401r 5.8298243e-04 3.15e+07 1.77e+03   1.0 1.39e+00   3.3 8.95e-01 1.29e-01f  1
1402r 5.8295165e-04 3.15e+07 1.97e+03   1.0 5.15e+00   2.8 6.78e-03 2.34e-02f  1
1403r 5.8288516e-04 3.15e+07 2.14e+03   1.0 8.22e-01   3.3 9.18e-02 1.34e-01f  1
1404r 5.8287439e-04 3.14e+07 2.62e+03   1.0 1.08e+01   2.8 3.28e-03 7.27e-03f  1
1405r 5.8282006e-04 3.14e+07 1.79e+03   1.0 6.37e-01   3.2 1.24e-01 9.76e-02f  1
1406r 5.8271113e-04 3.15e+07

1493r 5.7902790e-04 3.15e+07 1.62e+07   1.0 5.41e-02   7.2 3.15e-01 9.96e-01f  1
1494r 5.7902789e-04 3.15e+07 1.36e+07   1.0 1.75e-02   7.7 4.61e-01 1.75e-01f  1
1495r 5.7902789e-04 3.15e+07 1.12e+07   1.0 8.57e-03   8.1 1.00e+00 1.89e-01f  1
1496r 5.7902787e-04 3.15e+07 5.41e+06   1.0 1.24e-02   7.6 1.00e+00 1.00e+00f  1
1497r 5.7902780e-04 3.15e+07 3.27e+06   1.0 1.84e-02   7.1 6.06e-01 1.00e+00f  1
1498r 5.7902779e-04 3.15e+07 1.80e+06   1.0 4.86e-03   7.6 4.86e-01 5.39e-01f  1
1499r 5.7902771e-04 3.15e+07 3.60e+05   1.0 6.59e-03   7.1 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1500r 5.7902749e-04 3.15e+07 5.34e+04   1.0 1.27e-02   6.6 1.00e+00 1.00e+00f  1
1501r 5.7902681e-04 3.15e+07 4.09e+03   1.0 2.19e-03   6.1 1.00e+00 1.00e+00f  1
1502r 5.7902478e-04 3.15e+07 1.04e+03   1.0 2.29e-03   5.7 1.00e+00 1.00e+00f  1
1503r 5.7901868e-04 3.15e+07 1.00e+03   1.0 6.61e-03   5.2 1.00e+00 1.00e+00f  1
1504r 5.7900038e-04 3.15e+07

1588r 5.7506106e-04 3.15e+07 1.84e+04   1.0 6.31e-01   4.3 3.15e-01 1.42e-01f  1
1589r 5.7505604e-04 3.15e+07 1.60e+04   1.0 1.81e-01   4.7 1.27e-01 2.97e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1590r 5.7503334e-04 3.15e+07 8.07e+04   1.0 1.45e-01   4.3 4.52e-01 4.49e-01f  1
1591r 5.7502449e-04 3.15e+07 3.96e+04   1.0 7.71e-02   4.7 9.52e-01 4.67e-01f  1
1592r 5.7497165e-04 3.15e+07 2.28e+04   1.0 6.21e-02   4.2 1.00e+00 9.29e-01f  1
1593r 5.7495031e-04 3.15e+07 5.93e+05   1.0 2.33e-02   4.6 1.00e+00 1.00e+00f  1
1594r 5.7494963e-04 3.15e+07 8.24e+05   1.0 3.23e-02   6.0 1.00e+00 6.79e-01f  1
1595r 5.7494663e-04 3.15e+07 7.40e+05   1.0 1.17e-01   5.5 1.00e+00 1.00e+00f  1
1596r 5.7494561e-04 3.15e+07 4.21e+05   1.0 3.89e-02   5.9 1.00e+00 9.07e-01f  1
1597r 5.7494224e-04 3.15e+07 2.19e+05   1.0 6.26e-02   5.4 1.00e+00 1.00e+00f  1
1598r 5.7493211e-04 3.15e+07 9.38e+04   1.0 6.94e-02   5.0 1.00e+00 1.00e+00f  1
1599r 5.7490173e-04 3.15e+07

1684r 5.6786184e-04 3.15e+07 4.81e+16   1.0 6.56e-03  18.6 1.00e+00 1.00e+00h  1
1685r 5.6786184e-04 3.15e+07 1.66e+17   1.0 3.46e-02  19.1 1.00e+00 1.25e-01h  4
1686r 5.6786184e-04 3.15e+07 1.66e+17   1.0 3.46e-02  19.5 0.00e+00 0.00e+00R  1
1687r 5.6786184e-04 3.15e+07 1.68e+17   1.0 8.47e-03  19.9 9.90e-01 3.91e-03h  9
1688r 5.6786184e-04 3.15e+07 1.68e+17   1.0 8.47e-03  19.4 0.00e+00 0.00e+00R  1
1689r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  19.9 9.96e-01 6.25e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1690r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  19.4 0.00e+00 0.00e+00R  1
1691r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  19.8 0.00e+00 0.00e+00R  1
1692r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  20.0 0.00e+00 0.00e+00R  1
1693r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  20.0 0.00e+00 0.00e+00R  1
1694r 5.6786184e-04 3.15e+07 2.77e+17   1.0 1.83e-02  20.0 0.00e+00 0.00e+00R  1
1695r 5.6786184e-04 3.15e+07

In [ ]:
opti.debug.show_infeasibilities()